# Circuit learning method for SQL: training and test data generation

This notebook contains the data preparation and generation for the circuit learning process. Running the this notebook produces the training and test data which can be found from the `data` folder. This code is represented for reproducibility reasons and there is no need to rerun it every time. The queries are simplified versions of Join Order Benchmark queries.

### PostgreSQL database preparation

Prerequisites:

1. You need to have PostgreSQL database installed and configured
2. You have downloaded the old but more comprehensive IMDB data set from year 2017. See the instructions and link to the files from [cinemagoer](https://cinemagoer.readthedocs.io/en/latest/usage/ptdf.html).
3. Create an empty database (imdb2017) for the user (postgres) with the following code:

```
CREATE DATABASE imdb2017
WITH OWNER postgres
TEMPLATE = template0
ENCODING UTF8
LC_COLLATE = 'und-x-icu'
LC_CTYPE = 'und-x-icu';
```

IMDB is providing newer but less comprehensive [dataset](https://www.imdb.com/interfaces/). Cinemagoer describes how to upload that data into Postgres but because the join order benchmark queries are built on the older dataset, we also need to use the older one.

In [1]:
# Database credentials

port = "5432"
pg_db_name = "imdb2017"
pg_user = "postgres"
pg_pw = "0000"
imdb_file_path = "C:\\Users\\valte\\Documents\\frozendata"

pg_connection = "postgresql://" + pg_user + ":" + pg_pw + "@localhost:" + port + "/" + pg_db_name

The following code will use `imdbpy2sql` code to insert the internet movie database data into the created database. Note that the execution will probably take hours. Once the insert has been performed, comment the cell.

In [2]:
#!python cinemagoer\bin\imdbpy2sql.py -d {imdb_file_path} -u {pg_connection}

To speedup queries we create certain indeces.

In [3]:
import psycopg2

create_indices = """
create index company_id_movie_companies on movie_companies(company_id);
create index company_type_id_movie_companies on movie_companies(company_type_id);
create index info_type_id_movie_info on movie_info(info_type_id);
create index info_type_id_person_info on person_info(info_type_id);
create index keyword_id_movie_keyword on movie_keyword(keyword_id);
create index kind_id_aka_title on aka_title(kind_id);
create index kind_id_title on title(kind_id);
create index linked_movie_id_movie_link on movie_link(linked_movie_id);
create index link_type_id_movie_link on movie_link(link_type_id);
create index movie_id_aka_title on aka_title(movie_id);
create index movie_id_cast_info on cast_info(movie_id);
create index movie_id_complete_cast on complete_cast(movie_id);
create index movie_id_movie_companies on movie_companies(movie_id);
create index movie_id_movie_keyword on movie_keyword(movie_id);
create index movie_id_movie_link on movie_link(movie_id);
create index movie_id_movie_info on movie_info(movie_id);
create index person_id_aka_name on aka_name(person_id);
create index person_id_cast_info on cast_info(person_id);
create index person_id_person_info on person_info(person_id);
create index person_role_id_cast_info on cast_info(person_role_id);
create index role_id_cast_info on cast_info(role_id);
"""

test_query = "EXPLAIN ANALYZE SELECT * FROM title LIMIT 2;"

try:
    connection = psycopg2.connect(user=pg_user, password=pg_pw, host="localhost", port=port, database=pg_db_name)
    cursor = connection.cursor()
    cursor.execute(create_indices)
    res = cursor.fetchall()
    print(res)

except (Exception, psycopg2.Error) as error:
    print("Error while fetching data from PostgreSQL", error)

finally:
    # closing database connection.
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

Error while fetching data from PostgreSQL relation "company_id_movie_companies" already exists

PostgreSQL connection is closed


## Create training and test data

Training and test data are stored to `data` folder in json files.

In [4]:
# Read queries from the join order benchmark
import glob
import os
import re
import json
from pathlib import Path

this_folder = os.path.abspath(os.getcwd())
training_queries, validation_queries, test_queries = [], [], []
query_path_JOB = glob.glob(this_folder + "\\join-order-benchmark-queries\\[0-9]*.sql")
query_path_training = glob.glob(this_folder + "\\queries\\training_queries\\[0-9]*.sql")
query_path_validation = glob.glob(this_folder + "\\queries\\validation_queries\\[0-9]*.sql")
query_path_test = glob.glob(this_folder + "\\queries\\test_queries\\[0-9]*.sql")

for i, query in enumerate(query_path_training):
    base_name = Path(query).stem
    f = open(query, "r")
    training_queries.append({ 'name': base_name, 'query': f.read() })
    
for i, query in enumerate(query_path_validation):
    base_name = Path(query).stem
    f = open(query, "r")
    validation_queries.append({ 'name': base_name, 'query': f.read() })
        
for i, query in enumerate(query_path_test):
    base_name = Path(query).stem
    f = open(query, "r")
    test_queries.append({ 'name': base_name, 'query': f.read() })
        
print("Number of training queries is ", len(training_queries))
print("Number of validation queries is ", len(validation_queries))
print("Number of test queries is ", len(test_queries))

Number of training queries is  1106
Number of validation queries is  277
Number of test queries is  276


### Generating training and test data

In [5]:
def genereta_data(queries, ty = "training"):
    connection = psycopg2.connect(user=pg_user, password=pg_pw, host="localhost", port=port, database=pg_db_name)
    cursor = connection.cursor()
    cursor.execute("SET statement_timeout = 20000; COMMIT;")
    shots_per_query = 10
    data = []
    file_name = "data\\training_data.json"
    root_name = "training_data"

    for query in queries:
        try:
            total_running_time = 0.0
            for i in range(shots_per_query):
                cursor = connection.cursor()
                cursor.execute("EXPLAIN ANALYZE " + query['query'])
                res = cursor.fetchall()
                ex_time = float(re.findall("\d+\.\d+", res[-1][0])[0])
                plan_time = float(re.findall("\d+\.\d+", res[-2][0])[0])
                total_running_time += ex_time + plan_time
            #print(total_running_time / shots_per_query)
            data.append( {'name': query['name'], 'time': round(total_running_time / shots_per_query, 4) })

        except (Exception, psycopg2.Error) as error:
            print("Error while fetching data from PostgreSQL", error)
            print(query)

    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")
    
    if ty == "test":
        file_name = "data\\test_data.json"
        root_name = "test_data"
    elif ty == "validation":
        file_name = "data\\validation_data.json"
        root_name = "validation_data"
        
    with open(file_name, 'w') as outfile:
        json.dump({ root_name: data }, outfile)

In [ ]:
genereta_data(training_queries, "training")
genereta_data(validation_queries, "validation")
genereta_data(test_queries, "test")

Error while fetching data from PostgreSQL canceling statement due to statement timeout

{'name': '1001', 'query': "SELECT MIN(mi.info) AS release_date FROM aka_name AS an, cast_info AS ci, movie_info AS mi WHERE mi.info IN ('Sweden', 'USA', 'German') AND ci.movie_id = mi.movie_id AND an.person_id = ci.person_id;"}
Error while fetching data from PostgreSQL canceling statement due to statement timeout

{'name': '1003', 'query': "SELECT MIN(mc.note) AS production_note FROM cast_info AS ci, movie_companies AS mc, movie_info AS mi WHERE mc.note LIKE '%(France)%' AND mc.movie_id = mi.movie_id AND ci.movie_id = mi.movie_id;"}
Error while fetching data from PostgreSQL canceling statement due to statement timeout

{'name': '1021', 'query': "SELECT MIN(mc.note) AS production_note FROM movie_info AS mi, movie_companies AS mc, movie_keyword AS mk WHERE mc.note LIKE '%(France)%' AND mc.movie_id = mk.movie_id AND mc.movie_id = mi.movie_id;"}
Error while fetching data from PostgreSQL canceling statem